In [1]:
import os
import numpy as np
import scipy.io as sio
from scipy.optimize import minimize
import numpy as np
from scipy.optimize import minimize, Bounds
import matplotlib.pyplot as plt


#PART 1
# Function to load reaction time data from a .mat file
def load_reaction_time_data(subject_id=None, data_folder=None, express_cutoff_value=0.2):
    """
    Load reaction time data for a given subject and apply an express saccade cutoff.

    Parameters:
    subject_id : str, optional
        ID of the subject
    data_folder : str, optional
        Path to the folder containing data files (default: a specified directory).
    express_cutoff_value : float
        Minimum reaction time cutoff to filter out very fast responses (default: 0.2 seconds).

    Returns:
    tuple:
        - rt_data : list
            The loaded reaction time data.
        - trial_labels : list
            Associated labels or metadata for the trials.
    """

    # If no subject_id is provided, use a default one
    if subject_id is None:
        subject_id = 'RT'

    # If no data_folder is provided, set a default folder path
    if data_folder is None:
        data_folder = '/Users/cabi/Library/CloudStorage/Box-Box/QNC/LATERdata1'

    # Path to the .mat file that contains reaction time data
    file_path = os.path.join(data_folder, 'data_mgl', 'F', f'{subject_id}_RT.mat')

    try:
        # Load the .mat file containing the reaction time data
        loaded_data = sio.loadmat(file_path)
    except FileNotFoundError:
        raise FileNotFoundError(f"Data file not found at: {file_path}")

    # Extract the reaction time data and labels from the loaded .mat file
    # These variable names ('data_variable_name', 'labels_variable_name') should be updated to match your actual data
    rt_data = loaded_data.get('data_variable_name')  # Replace with actual variable name
    trial_labels = loaded_data.get('labels_variable_name')  # Replace with actual variable name

    return rt_data, trial_labels

# Function to define which trials to keep based on correctness and reaction time limits
def filter_trials(correct_trials, reaction_times, express_cutoff_value=0.2):
    """
    Filter trials based on correctness and reaction time limits.

    Parameters:
    correct_trials : np.ndarray
        Array indicating whether trials are correct (1 for correct, 0 for incorrect).
    reaction_times : np.ndarray
        Array of reaction times (RTs) for the trials.
    express_cutoff_value : float
        Minimum RT cutoff to remove very fast 'express' responses.

    Returns:
    np.ndarray
        A boolean array where True indicates the trial meets the selection criteria.
    """

    # Create a filter for valid trials: correct trials and reaction times within specified limits
    valid_trials = (correct_trials == 1) & (reaction_times > express_cutoff_value) & (reaction_times < 1.2)

    return valid_trials

# Function to categorize trials into different data sets based on the trial conditions
def categorize_trials(reaction_times, valid_trials, direction_choices, change_point_labels, return_labels=False):
    """
    Organize reaction times into different categories based on trial conditions.

    Parameters:
    reaction_times : np.ndarray
        Array of reaction times.
    valid_trials : np.ndarray
        Boolean array indicating which trials passed the filter.
    direction_choices : np.ndarray
        Array indicating whether the choice was left (-1) or right (1).
    change_point_labels : np.ndarray
        Array indicating if a trial was a change-point trial (1 for CP, 0 for no CP).
    return_labels : bool, optional
        If True, returns descriptive labels for the categorized data (default: False).

    Returns:
    list
        A list of arrays containing reaction times for each category of trial.
    list (optional)
        Descriptive labels for each category if return_labels is True.
    """

    categorized_data = []

    # Left choice, change-point trials (CP)
    categorized_data.append(reaction_times[valid_trials & (direction_choices == -1) & (change_point_labels == 1)])

    # Left choice, non-change-point trials (No CP)
    categorized_data.append(reaction_times[valid_trials & (direction_choices == -1) & (change_point_labels != 1)])

    # Right choice, change-point trials (CP)
    categorized_data.append(reaction_times[valid_trials & (direction_choices == 1) & (change_point_labels == 1)])

    # Right choice, non-change-point trials (No CP)
    categorized_data.append(reaction_times[valid_trials & (direction_choices == 1) & (change_point_labels != 1)])

    if return_labels:
        # Return the labels for the categories if requested
        labels = ['Left Choice, CP', 'Left Choice, No CP', 'Right Choice, CP', 'Right Choice, No CP']
        return categorized_data, labels

    return categorized_data


In [2]:
#PART 2
def calculate_negative_log_likelihood(rt_data, model_parameters):
    """
    Calculate the negative log-likelihood for reaction time data given model parameters.

    The objective is to find the model parameters that minimize this value,
    which corresponds to the maximum likelihood estimate for the data.

    Parameters:
    rt_data : np.ndarray
        Array of reaction times from the trials.
    model_parameters : dict
        Dictionary containing the model parameters:
        - 'mu': Mean of the reaction time distribution (mu = muR / deltaS).
        - 'sigma': Standard deviation of the reaction time distribution (sigma = 1 / deltaS).

    Returns:
    float
        The negative log-likelihood value for the given data and model parameters.
    """

    # Initialize an array to store the likelihood for each reaction time
    likelihood_values = np.zeros_like(rt_data)

    # Loop over each reaction time and compute its likelihood
    for i, rt in enumerate(rt_data):
        # Extract the mean and standard deviation from the model parameters
        mean_rt = model_parameters.get('mu', 0)  # Mean of the reaction time (mu)
        std_dev_rt = model_parameters.get('sigma', 1)  # Standard deviation (sigma)

        # Compute the likelihood of this reaction time given a normal distribution
        likelihood_values[i] = (1 / (std_dev_rt * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((rt - mean_rt) / std_dev_rt) ** 2)

    # Take the logarithm of the likelihoods (adding a small value to avoid log(0))
    log_likelihood_values = np.log(likelihood_values + 1e-10)

    # Sum all the log-likelihoods across the data
    total_log_likelihood = np.sum(log_likelihood_values)

    # The objective function is the negative of the total log-likelihood (to minimize it)
    negative_log_likelihood = -total_log_likelihood

    return negative_log_likelihood

#Tbh Eren I do not have the bandwidth to continue, I hope you understand.
#I figured submitting something was better than nothing. The NSF is due tomorrow and that is my priority right now.